In [ ]:
import os
# os.system('pip install pytorch_toolbelt')
import pandas as pd
import numpy as np
import json
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
# import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import time

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
# from sklearn.model_selection import KFold
import gc

from tqdm import tqdm
from itertools import groupby, accumulate
from random import shuffle

In [ ]:
from sklearn.model_selection import GroupKFold, GroupShuffleSplit, LeaveOneGroupOut
from sklearn.preprocessing import MinMaxScaler
# from pytorch_toolbelt import losses as L

In [ ]:
seed_val=100
# numpy RNG
np.random.seed(seed_val)

# pytorch RNGs
torch.manual_seed(seed_val)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed_val)

# python RNG
import random
random.seed(seed_val)

In [ ]:
# ss = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})
train = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv')
test = pd.read_csv('../input/data-without-drift/test_clean.csv')

### EDA

In [ ]:
plt.figure(figsize=(20,5)); res = 1000
plt.plot(range(0,train.shape[0],res),train.signal[0::res])
for i in range(11): plt.plot([i*500000,i*500000],[-5,12.5],'r')
for j in range(10): plt.text(j*500000+200000,10,str(j+1),size=20)
plt.xlabel('Row',size=16); plt.ylabel('Signal',size=16); 
plt.title('Training Data Signal - 10 batches',size=20)
plt.show()

### Model association

In [ ]:
train.loc[train.index<500000*2,'model']='1s'
train.loc[((train.index>=500000*2)&(train.index<500000*3))|((train.index>=500000*6)&(train.index<500000*7)),'model']='1f'
train.loc[((train.index>=500000*3)&(train.index<500000*4))|((train.index>=500000*7)&(train.index<500000*8)),'model']='3'
train.loc[((train.index>=500000*5)&(train.index<500000*6))|((train.index>=500000*8)&(train.index<500000*9)),'model']='5'
train.loc[((train.index>=500000*4)&(train.index<500000*5))|((train.index>=500000*9)&(train.index<500000*10)),'model']='10'

In [ ]:
train.model.value_counts(dropna=False)

In [ ]:
test.loc[((test.index>=100000*0)&(test.index<100000*1))|((test.index>=100000*3)&(test.index<100000*4)),'model']='1s'
test.loc[((test.index>=100000*8)&(test.index<100000*9))|((test.index>=100000*10)&(test.index<100000*20)),'model']='1s'
test.loc[((test.index>=100000*4)&(test.index<100000*5)),'model']='1f'
test.loc[((test.index>=100000*1)&(test.index<100000*2))|((test.index>=100000*9)&(test.index<100000*10)),'model']='3'
test.loc[((test.index>=100000*2)&(test.index<100000*3))|((test.index>=100000*6)&(test.index<100000*7)),'model']='5'
test.loc[((test.index>=100000*5)&(test.index<100000*6))|((test.index>=100000*7)&(test.index<100000*8)),'model']='10'

In [ ]:
test.model.value_counts(dropna=False)
# train.groupby(['model','open_channels']).size()

In [ ]:
a=train.groupby(['model','open_channels']).signal.agg(['mean','std']).reset_index()
a.sort_values(['model','open_channels'],inplace=True)
a['mean_diff']=a['mean']-a['mean'].shift(1)

In [ ]:
a.loc[a.model=='10','std']=0.413
a.loc[a.model=='1f','std']=0.2479
a.loc[a.model=='1s','std']=0.25
a.loc[a.model=='3','std']=0.507
a.loc[a.model=='5','std']=0.2909

# -4.193538-1.2336295466666667
a.loc[(a.model=='10')&(a.open_channels==0),'mean']=-5.4271675466666665

a['mean_diff']=a['mean']-a['mean'].shift(1)
a.loc[a.open_channels==0,'mean_diff']=np.nan

In [ ]:
a

In [ ]:
a['mean_diff'].values

In [ ]:
train['pulse_id']=((train.index//500000)+1).astype('int')

In [ ]:
train.groupby(['model','pulse_id','open_channels']).signal.agg(['mean','std'])

In [ ]:
for mod in ['10', '1f', '1s', '3', '5']:
    for chann in train.loc[train.model==mod,'open_channels'].unique():
        train.loc[train.model==mod , 
                  str(chann)+'_zscore']=(train.loc[train.model==mod ,
                                                   'signal']-a.loc[((a.model==mod)&(a.open_channels==chann)),
                                                                   'mean'].unique()[0])/(a.loc[(a.model==mod)&(a.open_channels==chann),'std'].unique()[0])

In [ ]:
for mod in ['10', '1f', '1s', '3', '5']:
    for chann in train.loc[train.model==mod,'open_channels'].unique():
        test.loc[test.model==mod , 
                  str(chann)+'_zscore']=(test.loc[test.model==mod ,
                                                   'signal']-a.loc[((a.model==mod)&(a.open_channels==chann)),
                                                                   'mean'].unique()[0])/(a.loc[(a.model==mod)&(a.open_channels==chann),'std'].unique()[0])

In [ ]:
zscore_cols=[str(i) +'_zscore' for i in range(11)]

In [ ]:
train[zscore_cols]=train[zscore_cols].abs()
test[zscore_cols]=test[zscore_cols].abs()
train=train.fillna(1000)
test=test.fillna(1000)

In [ ]:
def using_indexed_assignment(x,n_):
    "https://stackoverflow.com/a/5284703/190597 (Sven Marnach)"
    result = np.empty(len(x), dtype=int)
    temp = x
    result[temp] = np.arange(len(x))
    result= list(result)
    return result.index(n_)

In [ ]:
train['dist_target']=[using_indexed_assignment(i,0) for i in np.argsort(train[zscore_cols].values)]
test['dist_target']=[using_indexed_assignment(i,0) for i in np.argsort(test[zscore_cols].values)]

In [ ]:
train['dist_target_2']=[using_indexed_assignment(i,1) for i in np.argsort(train[zscore_cols].values)]
test['dist_target_2']=[using_indexed_assignment(i,1) for i in np.argsort(test[zscore_cols].values)]

In [ ]:
f1_score(train.open_channels,train.dist_target,average='macro')

In [ ]:
for i in train['model'].unique():
    print(i,f1_score(train[train.model==i].open_channels,train[train.model==i].dist_target,average='macro'))

In [ ]:
train[train.model=='10'].open_channels.value_counts()

In [ ]:
a1=train[(train.model=='10')&(train.pulse_id==5)].open_channels.values
a2=train[(train.model=='10')&(train.pulse_id==10)].open_channels.values
len(a1),len(a2)

In [ ]:
def transition_matrix(transitions):
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

#test:

# t = [1,1,2,6,8,5,5,7,8,8,1,1,4,5,5,0,0,0,1,1,4,4,5,1,3,3,4,5,4,1,1]
# m = transition_matrix(t)
# for row in m: print(' '.join('{0:.2f}'.format(x) for x in row))

In [ ]:
m1=transition_matrix(a1)
m2=transition_matrix(a2)
m=np.mean([m1,m2],axis=0)

In [ ]:
# m10=train[train.model=='10']
# e=m10[m10.dist_target!=m10.open_channels][['open_channels','dist_target']]
# e['comb_']=e.open_channels.astype('str')+'_'+e.dist_target.astype('str')
# e['comb_'].value_counts()

In [ ]:
# m10=train[train.model=='10']
# e=m10[m10.dist_target!=m10.open_channels]
# e['delta_']=e.apply(lambda x :x[str(x['dist_target_2'])+'_zscore']-x[str(x['dist_target'])+'_zscore'],axis=1)
# e['delta_'].describe()

In [ ]:
# m10=train[train.model=='10']
# e=m10[m10.dist_target==m10.open_channels]
# e['delta_']=e.apply(lambda x :x[str(x['dist_target_2'])+'_zscore']-x[str(x['dist_target'])+'_zscore'],axis=1)
# e['delta_'].describe()

---

In [ ]:
train['dist_target_col']=train.dist_target.astype('str')+'_zscore'
train['dist_target_2_col']=train.dist_target_2.astype('str')+'_zscore'
train['delta_']=train.apply(lambda x :x[x['dist_target_2_col']]-x[x['dist_target_col']],axis=1)

In [ ]:
test['dist_target_col']=test.dist_target.astype('str')+'_zscore'
test['dist_target_2_col']=test.dist_target_2.astype('str')+'_zscore'
test['delta_']=test.apply(lambda x :x[x['dist_target_2_col']]-x[x['dist_target_col']],axis=1)

In [ ]:
q=0.8

In [ ]:
(train[train.delta_<=q].dist_target==train[train.delta_<=q].open_channels).value_counts(1),f1_score(train[train.delta_<=q].open_channels,
                                                                                                   train[train.delta_<=q].dist_target,average='macro')

### on train

In [ ]:
%%time
import copy
idx_train=pd.Series([m for j in [[i,i-1,i+1] for i in train[train.delta_<=q].index] for m in j]).sort_values().unique()
e=train.iloc[idx_train,:]
e['dist_target_last']=e['dist_target'].shift(1)
e['dist_target_next']=e['dist_target'].shift(-1)
e=e.fillna(0)
for i in ['dist_target','dist_target_2','dist_target_last','dist_target_next']:
    e[i]=e[i].astype('int')
# e.shape,len(idx_train),train[train.delta_<=q].shape
e['prob_1n']=e.apply(lambda x: m[x['dist_target'],x['dist_target_next']],axis=1)
e['prob_2n']=e.apply(lambda x: m[x['dist_target_2'],x['dist_target_next']],axis=1)
e['prob_l1']=e.apply(lambda x: m[x['dist_target_last'],x['dist_target']],axis=1)
e['prob_l2']=e.apply(lambda x: m[x['dist_target_last'],x['dist_target_2']],axis=1)
e['prob_1']=e.prob_1n*e.prob_l1
e['prob_2']=e.prob_2n*e.prob_l2
e['is_greater_12']=(e['prob_1']>e['prob_2']).astype('int')
e['dist_dummy']=copy.deepcopy(e.dist_target)
e.loc[((e['is_greater_12']==0)&(e.delta_<=q)),'dist_dummy']=e.loc[((e['is_greater_12']==0)&(e.delta_<=q)),
                                                                  'dist_target_2']

In [ ]:
f1_score(e.open_channels,e.dist_target,average='macro'),f1_score(e.open_channels,e.dist_dummy,average='macro')

In [ ]:
train=train.merge(e[['time','dist_dummy']],on='time',how='left')
train.loc[((train['dist_dummy'].notnull())&(train.model.isin(['10','5']))),
          'dist_target']=train.loc[((train['dist_dummy'].notnull())&(train.model.isin(['10','5']))),'dist_dummy']

In [ ]:
f1_score(train.open_channels,train.dist_target,average='macro')

In [ ]:
for i in train['model'].unique():
    print(i,f1_score(train[train.model==i].open_channels,train[train.model==i].dist_target,average='macro'))

### on test

In [ ]:
%%time
import copy
idx_test=pd.Series([m for j in [[i,i-1,i+1] for i in test[test.delta_<=q].index] for m in j]).sort_values().unique()
e=test.iloc[idx_test,:]
e['dist_target_last']=e['dist_target'].shift(1)
e['dist_target_next']=e['dist_target'].shift(-1)
e=e.fillna(0)
for i in ['dist_target','dist_target_2','dist_target_last','dist_target_next']:
    e[i]=e[i].astype('int')
print(e.shape,len(idx_test),test[test.delta_<=q].shape)
e['prob_1n']=e.apply(lambda x: m[x['dist_target'],x['dist_target_next']],axis=1)
e['prob_2n']=e.apply(lambda x: m[x['dist_target_2'],x['dist_target_next']],axis=1)
e['prob_l1']=e.apply(lambda x: m[x['dist_target_last'],x['dist_target']],axis=1)
e['prob_l2']=e.apply(lambda x: m[x['dist_target_last'],x['dist_target_2']],axis=1)
e['prob_1']=e.prob_1n*e.prob_l1
e['prob_2']=e.prob_2n*e.prob_l2
e['is_greater_12']=(e['prob_1']>e['prob_2']).astype('int')
e['dist_dummy']=copy.deepcopy(e.dist_target)
e.loc[((e['is_greater_12']==0)&(e.delta_<=q)),'dist_dummy']=e.loc[((e['is_greater_12']==0)&(e.delta_<=q)),
                                                                  'dist_target_2']

In [ ]:
(e.dist_target==e.dist_dummy).value_counts()

In [ ]:
test=test.merge(e[['time','dist_dummy']],on='time',how='left')
test.loc[((test['dist_dummy'].notnull())&(test.model.isin(['10','5']))),
          'dist_target']=test.loc[((test['dist_dummy'].notnull())&(test.model.isin(['10','5']))),'dist_dummy']

---

In [ ]:
group_size=1000

In [ ]:
train['group'] = train.groupby(train.index//group_size, sort=False)['signal'].agg(['ngroup']).values
folds=GroupKFold(n_splits=5)
folds=[(tr,val) for tr,val in folds.split(train,train['open_channels'],train['group'])]

In [ ]:
[(len(i[0]),len(i[1])) for i in folds]

In [ ]:
class EarlyStopping(object):
    def __init__(self,patience,higher_is_better,tolerance,save_path,model):
        self.patience=patience
        self.higher_is_better = higher_is_better
        self.best_score= -100000 if self.higher_is_better else 100000
        self.tolerance=tolerance ### score should inc/dec by this much to by pass early stopping
        self.ctr=0
        self.save_path=save_path
        self.model=model

    def check(self,metric):
        if self.ctr<self.patience:

            if self.higher_is_better:
                if metric>=(self.best_score+self.tolerance):
                    self.best_score=metric
                    self.ctr =0
                    torch.save(self.model.state_dict(), self.save_path)
                else:
                     self.ctr += 1
            else:
                if metric<=(self.best_score-self.tolerance):
                    self.best_score=metric
                    self.ctr =0
                    torch.save(self.model.state_dict(), self.save_path)
                else:
                     self.ctr += 1
            return True

        else:
            return False

In [ ]:
train.shape,test.shape

In [ ]:
cols_=[ 'signal', 
#         'model',
#        '10_zscore', '9_zscore',
#        '8_zscore', '7_zscore', '6_zscore', '5_zscore', '4_zscore', '3_zscore',
#        '2_zscore', '1_zscore', '0_zscore', 
       'dist_target']

In [ ]:
len(cols_)

In [ ]:
# a,b=next(iter(train_dataloader))
# a.shape,b.shape

In [ ]:
class IonDataset(Dataset):

    def __init__(self,X,y,seq_len=1000):

        self.seq_len=seq_len
        self.size_=int(X.shape[0]/seq_len)
        self.x=X.astype(np.float32).values

        if y is not None:
            self.y=y.values
        else:
            self.y=None

    def __len__(self):
        return self.size_

    def __getitem__(self,idx):
        if self.y is not None:
            return  self.x[idx*self.seq_len:(idx+1)*self.seq_len],\
                    self.y[idx*self.seq_len:(idx+1)*self.seq_len],\

        else:
            return self.x[idx*self.seq_len:(idx+1)*self.seq_len]

In [ ]:
class Waveblock(nn.Module):
    def __init__(self,num_layers,kernel_size,input_channels,dilation_dim,skip_conv_dim,res_conv_dim,output_channels):
        super(Waveblock,self).__init__()
        ###res conv dim and causal conv dim will be same so that they can be added in the residual block
        self.causal_conv=nn.Conv1d(input_channels,res_conv_dim,kernel_size=1)
        
        self.gate_convs=nn.ModuleList()
        self.filter_convs=nn.ModuleList()
        ##res scaling and skip scaling convs should not affect size of feature map
        self.res_scaling_convs=nn.ModuleList()
        self.skip_scaling_convs=nn.ModuleList()
        
        self.num_layers=num_layers
        
        for i in range(self.num_layers):
            dilation=2**i
            ##so that Lin==Lout
            pad_size=int(dilation*(kernel_size-1)/2)
            
            self.gate_convs.append(nn.Conv1d(res_conv_dim,dilation_dim,dilation=dilation,kernel_size=kernel_size,padding=pad_size))
            self.filter_convs.append(nn.Conv1d(res_conv_dim,dilation_dim,dilation=dilation,kernel_size=kernel_size,padding=pad_size))
            self.res_scaling_convs.append(nn.Conv1d(dilation_dim,res_conv_dim,kernel_size=1))
            self.skip_scaling_convs.append(nn.Conv1d(dilation_dim,skip_conv_dim,kernel_size=1))
        
        self.final_conv1=nn.Conv1d(skip_conv_dim,skip_conv_dim,kernel_size=1)
        self.final_conv2=nn.Conv1d(skip_conv_dim,output_channels,kernel_size=1)
        
    def forward(self,x):
        ###shape should be (batch,input channels,seq_len)
        x=self.causal_conv(x)
        
        for i in range(self.num_layers):
            #print('residual block:',i)
            filter_=self.filter_convs[i](x)
            gate_=self.gate_convs[i](x)
            ##element wise multiplication
            gated_act_=(nn.Tanh()(filter_))*(nn.Sigmoid()(gate_))
            #print(gated_act_.shape)
            #print(self.res_scaling_convs[i](gated_act_).shape,x.shape)
            x=torch.add(self.res_scaling_convs[i](gated_act_),x)
            ##this x will feed into next layer(residual block)
            
            skip_output=self.skip_scaling_convs[i](gated_act_)
            if i==0:
                final_output=skip_output
            else:
                final_output=torch.add(final_output,skip_output)
        
        final_output=nn.ReLU()(final_output)
        final_output=self.final_conv1(final_output)
        final_output=nn.ReLU()(final_output)
        final_output=self.final_conv2(final_output)
        
        ##final output shape will be (batch,output_channels,seq_len)
        ##for LSTM we need (batch,seq_len,input_dim),permute later to (0,2,1)
        return final_output

In [ ]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_rnn_layers,hidden_layers,bidirectional,dropout):
        
        super().__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.output_size=output_size
        self.num_rnn_layers=num_rnn_layers
        self.bidirectional=bidirectional
        self.dropout=dropout
        self.len_hidden_layers=isinstance(hidden_layers,list)
        
        self.rnn=nn.GRU(batch_first=True,input_size=self.input_size,hidden_size=self.hidden_size,bidirectional=self.bidirectional,
                         num_layers=self.num_rnn_layers,dropout=self.dropout)
        
        #Input layer
        
        if hidden_layers:
            first_layer=nn.Linear(self.hidden_size*2 if self.bidirectional else self.hidden_size,hidden_layers[0])
        
        #Hidden layers
        if self.len_hidden_layers:
            self.hidden_layers=nn.ModuleList( [first_layer] + [nn.Linear(hidden_layers[i],hidden_layers[i+1]) for i in range(len(hidden_layers)-1)])
            for layer in self.hidden_layers: nn.init.kaiming_normal_(layer.weight.data)
        else:
            self.hidden_layers=first_layer
            nn.init.kaiming_normal_(self.hidden_layers.weight.data)
            
        #Output layer
        
        self.output_layer=nn.Linear(hidden_layers[-1],self.output_size)
        nn.init.kaiming_normal_(self.output_layer.weight.data)
        
        self.activation_fn=torch.relu
        self.dropout=nn.Dropout(dropout)
    
    def forward(self,x):
#         print(x.shape)
        
#         batch_size=x.size(0)
        
        #print('input before permute shape: ',x.shape)
        
        ### use permute when input dimension more than 1
        
#         x=x.view(x.shape[0],x.shape[-1],1)
        
        #print('input to model shape: ',x.shape)
        
        ###shape should be (batch,seq_en,input_dim)
        
        outputs,hidden=self.rnn(x)
        
#         print(outputs.shape)
        x=self.dropout(self.activation_fn(outputs))
        
        if self.len_hidden_layers:
            for layer in self.hidden_layers:
                x = self.activation_fn(layer(x))
                x = self.dropout(x)
        else:
            x = self.activation_fn(self.hidden_layers(x))
            x = self.dropout(x)
        
        x=self.output_layer(x)
        
        return x

In [ ]:
# wavenet=Waveblock(num_layers=10,kernel_size=2,input_channels=1,dilation_dim=128,
#                       skip_conv_dim=128,
#                       res_conv_dim=24,
#                       output_channels=3)
# rnn = RNN(input_size=3,hidden_size=128,output_size=11,num_rnn_layers=3,
#           hidden_layers=[512,64],bidirectional=True,dropout=0.3)

In [ ]:
class model_(nn.Module):
    def __init__(self,wavenet_dict,rnn_dict):
        super().__init__()
        self.waveblock=Waveblock(**wavenet_dict)
        self.rnn=RNN(**rnn_dict)
    def forward(self,x):
        ###shape should be (batch,input channels,seq_len)
        #print('before permute shape:',x.shape)
        #x=x.view(x.shape[0],1,x.shape[-1])
        x=x.permute(0,2,1)
        #print('before model shape:',x.shape)
        x=self.waveblock(x)
        #print('after waveblock shape:',x.shape)
        x=x.permute(0,2,1)
        ###shape should be (batch,seq_len,input dim)
        #print('before rnn shape:',x.shape)
        x=self.rnn(x)
        return x

In [ ]:
len(folds),len(folds[:3])

In [ ]:
def init_state_dict():
    return {'model_dict':None,'epoch_num':0,'batch_num':0,'train_params':{'avg_loss':[],'metric':[]},
            'valid_params':{'avg_loss':[],'metric':[]}}

def get_F1_score(true_list,preds_list):
    return f1_score(np.concatenate(true_list),np.concatenate(preds_list),average='macro')

def validate_loop(valid_dl,model,loss_fn,model_dict,beta=0.98):

    model.eval()
    with torch.no_grad():
        loss_=0
        pred_list=[]
        tar_list=[]
    for i,(num,tar) in enumerate(valid_dl):
            num = num.to(device)
            tar = tar.to(device)
            preds = model(num)
            preds=preds.view(-1,preds.shape[-1])
            tar=tar.view(-1)
            loss = loss_fn(preds, tar)
            loss_ += loss.item()
            pred_list.append(preds.cpu().detach().numpy().argmax(1))
            tar_list.append(tar.cpu().detach().numpy())
        
    model_dict['valid_params']['avg_loss'].append(loss_/i)
    model_dict['valid_params']['metric'].append(get_F1_score(tar_list,pred_list))

    return model_dict

def train_loop(model,train_dl,valid_dl,num_epoch,optimizer,loss_fn,schedular,early_stopping,log_file,beta=0.98):

    model_dict=init_state_dict()

    for j,epoch in enumerate(range(num_epoch)):
        st=time.time()

        model.train()

        loss_=0
        pred_list=[]
        tar_list=[]

        for i, (num,tar) in enumerate(train_dl):

            num = num.to(device)
            tar = tar.to(device)
            optimizer.zero_grad()
            
            preds = model(num)
            preds=preds.view(-1,preds.shape[-1])
            tar=tar.view(-1)
            loss = loss_fn(preds, tar)
            loss.backward()
            optimizer.step()
            
            if schedular is not None:
                schedular.step()
            loss_ += loss.item()
            model_dict['batch_num']+=1
            
            pred_list.append(preds.cpu().detach().numpy().argmax(1))
            tar_list.append(tar.cpu().detach().numpy())

        model_dict['train_params']['avg_loss'].append(loss_/i)
        model_dict['train_params']['metric'].append(get_F1_score(tar_list,pred_list))

        model_dict['epoch_num']=j

        model_dict=validate_loop(valid_dl=valid_dl,model=model,loss_fn=loss_fn,model_dict=model_dict,beta=beta)

        val_metric=model_dict['valid_params']['metric'][j]
        
#         if schedular is not None:
#             schedular.step(metrics = val_metric)
            
        et=time.time()
        if not early_stopping.check(val_metric):
            return model_dict
        
        mssg = 'Epoch {}: train_loss:{} valid_loss:{} train_metric:{} valid_metric:{} time:{}'.format(model_dict['epoch_num'],
                                                    round(model_dict['train_params']['avg_loss'][j],4),
                                                    round(model_dict['valid_params']['avg_loss'][j],4),
                                                    round(model_dict['train_params']['metric'][j],4),
                                                    round(model_dict['valid_params']['metric'][j],4),round((et-st)/60,2))
        
#         slack_loggger('GTX1660 LSTM LIVERPOOL: ',mssg)
        print (mssg)
#         f= open(log_file, 'a')  
#         f.write(mssg)
#         f.close()

        # model_dict['model_dict']=model.state_dict() ### keep updating the model dict if it doesn't meet early stopping
    return model_dict

In [ ]:
SEQ_LEN = 1000
IS_PRETRAINED = False
NUM_WORKERS = 4
batchsize=32
device=torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
train.shape

In [ ]:
train['open_channels']=train['open_channels'].astype('int')

In [ ]:
wavenet_dict={'num_layers':8,'kernel_size':3,'input_channels':2,'dilation_dim':32,
              'skip_conv_dim':32,'res_conv_dim':32,'output_channels':32}
rnn_dict={'input_size':32,'hidden_size':64,'output_size':11,'num_rnn_layers':2,
          'hidden_layers':[128],'bidirectional':True,'dropout':0.2}

In [ ]:
IS_PRETRAINED = False

---

### pretrained score check

In [ ]:
# def validate_loop_pretrained(valid_dl,model,loss_fn,beta=0.98):

#     model.eval()
#     with torch.no_grad():
#         loss_=0
#         pred_list=[]
#         tar_list=[]
#     for i,(num,tar) in enumerate(valid_dl):
#             num = num.to(device)
#             tar = tar.to(device)
#             preds = model(num)
#             preds=preds.view(-1,preds.shape[-1])
#             tar=tar.view(-1)
#             loss = loss_fn(preds, tar)
#             loss_ += loss.item()
#             pred_list.append(preds.cpu().detach().numpy().argmax(1))
#             tar_list.append(tar.cpu().detach().numpy())
        
#     print('loss',loss_/i,'metric',get_F1_score(tar_list,pred_list))
# #     model_dict['valid_params']['metric'].append()
#     return True

In [ ]:
def predict(test_dl,model,model_path=None):

    if model_path is not None:
        model.load_state_dict(torch.load(model_path))

    model.eval()

    with torch.no_grad():
        pred_list=[]
        id_list=[]
    for i,num in enumerate(test_dl):
            num = num.to(device)
#             id_ = id_.to(device)
#             id_=id_.view(-1)
            preds=model(num)
            preds=preds.view(-1,preds.shape[-1])
            pred_list.append(preds.cpu().detach().numpy())
#             print(preds.shape)
#             id_list.append(id_.cpu().detach().numpy())

#     out_ =np.concatenate(pred_list),np.concatenate(id_list)
#     out_ =np.concatenate(pred_list)    
    
    return np.vstack(pred_list)

In [ ]:
# IS_PRETRAINED=True
# loss_fn=nn.CrossEntropyLoss()

In [ ]:
# test_df = IonDataset(X=test[cols_],y=None
#                      ,seq_len=SEQ_LEN)
# test_dl =DataLoader(test_df, batchsize, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
# # 
# preds = []
# for i in range(3):
      
# #     train_df = IonDataset(X=train[cols_].iloc[folds[i][0]],
# #                        seq_len=SEQ_LEN,y=train['open_channels'].iloc[folds[i][0]])
# #     train_dl = DataLoader(dataset=train_df,batch_size=batchsize,shuffle=True,drop_last=True,num_workers=NUM_WORKERS)

#     val_df = IonDataset(X=train[cols_].iloc[folds[i][1]],
#                        seq_len=SEQ_LEN,y=train['open_channels'].iloc[folds[i][1]])
#     val_dl = DataLoader(dataset=val_df,batch_size=batchsize,shuffle=True,num_workers=NUM_WORKERS)

#     rnn=model_(wavenet_dict,rnn_dict).to(device)    
#     if IS_PRETRAINED:
#         rnn.load_state_dict(torch.load('../input/ion-switchingmodels-v2/GRU_focal_folds_{}.path'.format(i)))
    
#     validate_loop_pretrained(val_dl,rnn,loss_fn,beta=0.98)
#     rnn=model_(wavenet_dict,rnn_dict).to(device)    
#     preds.append(predict(test_dl,rnn,'../input/ion-switchingmodels-v2/GRU_focal_folds_{}.path'.format(i)))

In [ ]:
np.mean([0.940227802425148,0.9407934877974964,0.939087731720544])

---

In [ ]:
IS_PRETRAINED=True

In [ ]:
for i in range(len(folds)):
      
    train_df = IonDataset(X=train[cols_].iloc[folds[i][0]],
                       seq_len=SEQ_LEN,y=train['open_channels'].iloc[folds[i][0]])
    train_dl = DataLoader(dataset=train_df,batch_size=batchsize,shuffle=True,drop_last=True,num_workers=NUM_WORKERS)

    val_df = IonDataset(X=train[cols_].iloc[folds[i][1]],
                       seq_len=SEQ_LEN,y=train['open_channels'].iloc[folds[i][1]])
    val_dl = DataLoader(dataset=val_df,batch_size=batchsize,shuffle=True,num_workers=NUM_WORKERS)

    rnn=model_(wavenet_dict,rnn_dict).to(device)
    if IS_PRETRAINED:
        rnn.load_state_dict(torch.load('../input/ionmodels5f-90ep/GRU_focal_folds_{}.path'.format(i)))

    max_lr=0.001
    early_stopping_rounds=40
    maximize=True
    num_epochs=50
    loss_fn=nn.CrossEntropyLoss()
#     loss_fn=FocalLoss(gamma=1)
    optimizer = torch.optim.AdamW(params=rnn.parameters(),lr=max_lr)

    schedular=torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer,max_lr=max_lr,epochs=num_epochs,
                                            steps_per_epoch=len(train_dl))
    
    es=EarlyStopping(patience=early_stopping_rounds,higher_is_better=1,tolerance=0.0001,
                     save_path='GRU_focal_folds_{}.path'.format(i),model=rnn)
    dict_=train_loop(model=rnn,train_dl = train_dl ,valid_dl = val_dl,log_file='GRU_log.txt',
                 num_epoch=num_epochs,optimizer=optimizer,loss_fn=loss_fn,
                     schedular=schedular,early_stopping=es,beta=0.98)
#     print('BEST SCORE FOR FOLD {} IS {}'.format(i,np.max(dict_['valid_params']['metric'])))

In [ ]:
# np.mean([93.95,94.07,93.82,93.92,93.94])

In [ ]:
(.9385+.9395+.9377+.9374+.9385)/5

In [ ]:
np.mean([0.9397932522542907,0.9404833125183752,0.9381518885864977,0.9388859828821762])

----

In [ ]:
%%time
test_df = IonDataset(X=test[cols_],y=None
                     ,seq_len=SEQ_LEN)
test_dl =DataLoader(test_df, batchsize, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
preds = []
for i in range(len(folds)):
    
    rnn=model_(wavenet_dict,rnn_dict).to(device)
#     rnn.load_state_dict(torch.load("GRU_focal_folds_{}.path".format(i)))
    preds.append(predict(test_dl,rnn,'GRU_focal_folds_{}.path'.format(i)))

In [ ]:
sub = pd.read_csv('/kaggle/input/liverpool-ion-switching/sample_submission.csv')

In [ ]:
sub['open_channels']=pd.Series(np.mean(preds,axis=0).argmax(axis=1))

In [ ]:
sub['open_channels'].iloc[-1000000:].value_counts()

In [ ]:
sub.to_csv('baseline_wavenet_GRU_3_layer_bidir.csv', index=False, float_format='%.4f')